In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor


from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, Activation, Conv2D, Dropout, Flatten
from keras import optimizers, utils, initializers, regularizers
import keras.backend as K

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler #Importing the StandardScaler

from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt

#Geo plotting libraries
import geopandas as gdp
from matplotlib.colors import ListedColormap
# import geoplot as glpt

import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import linear_model
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.model_selection import KFold
import matplotlib.lines as mlines

import statistics

from scipy.stats import rv_continuous
from scipy.stats import *

from statistics import mean
from sklearn.cluster import KMeans
from scipy.spatial import distance


######### Instance Transfer repositories ####################
from adapt.instance_based import TwoStageTrAdaBoostR2

print("Repositories uploaded!!")

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Second Upload Completed!!")

######### Importing required Libraries for Plotting the Map #########

import geopandas as gpd
import json
import altair as alt
import pandas as pd

from os import listdir
from os.path import isfile, join
import glob

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
import seaborn as sns

Repositories uploaded!!
Second Upload Completed!!


In [3]:
from dask.distributed import Client
client = Client()

import joblib

/Users/shrey/anaconda3/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 60749 instead
  warnings.warn(


In [5]:
######################################################## Automobile ################################################################
## horsepower column has correlation 0.4 :: [46 - 230] :: 30
#################################################################################################################################

dropcol_initial_auto = ['name']
AutoData_df = pd.read_csv('UCI_regression/MPG/Auto.csv') ## horsepower column has correlation 0.4 :: [46 - 230] :: 30
AutoData_df = AutoData_df.drop(dropcol_initial_auto, axis = 1)
print("The shape of the Input data is: ", AutoData_df.shape)

drop_col_auto = ['horsepower']

auto_tgt_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]
auto_tgt_df = auto_tgt_df.drop(drop_col_auto, axis = 1)
auto_tgt_df = auto_tgt_df.reset_index(drop=True)
print("Target Set: ",auto_tgt_df.shape)

auto_source_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]
auto_source_df = auto_source_df.drop(drop_col_auto, axis = 1)
auto_source_df = auto_source_df.reset_index(drop=True)
print("Source Set: ",auto_source_df.shape)

auto_test_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]
auto_test_df = auto_test_df.drop(drop_col_auto, axis = 1)
auto_test_df = auto_test_df.reset_index(drop=True)
print("Test Set: ",auto_test_df.shape)

#################### Splitting into features and target ####################
target_column_auto = ['mpg']

auto_tgt_df_y = auto_tgt_df[target_column_auto]
auto_tgt_df_X = auto_tgt_df.drop(target_column_auto, axis = 1)

auto_source_df_y = auto_source_df[target_column_auto]
auto_source_df_X = auto_source_df.drop(target_column_auto, axis = 1)

auto_test_df_y = auto_test_df[target_column_auto]
auto_test_df_X = auto_test_df.drop(target_column_auto, axis = 1)

############## Merging the datasets ##########################################
auto_X_df = pd.concat([auto_tgt_df_X, auto_source_df_X], ignore_index=True)
auto_y_df = pd.concat([auto_tgt_df_y, auto_source_df_y], ignore_index=True)

auto_np_train_X = auto_X_df.to_numpy()
auto_np_train_y = auto_y_df.to_numpy()

auto_np_test_X = auto_test_df_X.to_numpy()
auto_np_test_y = auto_test_df_y.to_numpy()

auto_np_train_y_list = auto_np_train_y.ravel()
auto_np_test_y_list = auto_np_test_y.ravel()

src_size_auto = len(auto_source_df_y)
tgt_size_auto = len(auto_tgt_df_y)

print("---------------------------")


#################################### STrAdaBoost.R2 Auto ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

sample_size = [len(auto_tgt_df_X), len(auto_source_df_X)]
n_estimators = 50
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_auto = []
rmselist_stradaboost_auto = []

for x in range(0, 1):

    model_stradaboost_auto = TwoStageTrAdaBoostR2(RandomForestRegressor(max_depth = 4, n_jobs = 5),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)

    with joblib.parallel_backend('dask'):
        model_stradaboost_auto.fit(auto_np_train_X, auto_np_train_y_list)
        y_pred_stradaboost_auto = model_stradaboost_auto.predict(auto_np_test_X)


    mse_stradaboost_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_stradaboost_auto))
    rmselist_stradaboost_auto.append(mse_stradaboost_auto)

    r2_score_stradaboost_auto = pearsonr(auto_np_test_y_list, y_pred_stradaboost_auto)
    r2_score_stradaboost_auto = (r2_score_stradaboost_auto[0])**2
    r2scorelist_stradaboost_auto.append(r2_score_stradaboost_auto)


print("RMSE of STrAdaboostR2:", statistics.mean(rmselist_stradaboost_auto))
print("R^2 of STrAdaboostR2:", statistics.mean(r2scorelist_stradaboost_auto))
print("\n")
print("RMSE of STrAdaboostR2:", rmselist_stradaboost_auto)
print("R^2 of STrAdaboostR2:", r2scorelist_stradaboost_auto)


print("-------------------------------------------")

The shape of the Input data is:  (392, 8)
Target Set:  (119, 7)
Source Set:  (157, 7)
Test Set:  (116, 7)
---------------------------
Inside STrAdaBoost.R2
RMSE of STrAdaboostR2: 6.828245498594654
R^2 of STrAdaboostR2: 0.5113371898216029


RMSE of STrAdaboostR2: [6.828245498594654]
R^2 of STrAdaboostR2: [0.5113371898216029]
-------------------------------------------


In [ ]:
######################### Gradient Boosting Regression Elevators #######################################
from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression")
print("-------------------------------------------")

r2scorelist_GBR_elevators = []
rmselist_GBR_elevators = []

for x in range(0, 10):

    model_GBR_elevators = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100)
    model_GBR_elevators.fit(Elevators_tgt_df_X, Elevators_tgt_df_y)

    y_pred_GBR_elevators = model_GBR_elevators.predict(elevators_np_test_X) ##Using dataframe instead of the numpy matrix

    mse_GBR_elevators = sqrt(mean_squared_error(elevators_np_test_y, y_pred_GBR_elevators))
    rmselist_GBR_elevators.append(mse_GBR_elevators)
        
    r2_score_GBR_elevators = pearsonr(elevators_np_test_y_list, y_pred_GBR_elevators)
    r2_score_GBR_elevators = (r2_score_GBR_elevators[0])**2
    r2scorelist_GBR_elevators.append(r2_score_GBR_elevators)


print("RMSE of GBR:", statistics.mean(rmselist_GBR_elevators))
print("R^2 of GBR:", statistics.mean(r2scorelist_GBR_elevators))
print("\n")
print("RMSE of GBR:", rmselist_GBR_elevators)
print("R^2 of GBR:", r2scorelist_GBR_elevators)

print("-------------------------------------------")